<img style="float: right; margin: 30px 15px 15px 15px;" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFzQj91sOlkeDFkg5HDbjtR4QJYmLXkfMNig&usqp=CAU" width="300" height="400" /> 
    
    
### <font color='navy'> Modelos no Lineales para Pronósticos. </font>

**Nombres:**
> `Cárdenas Gallardo Paula Daniela` | `733720` <br> `Haces López José Manuel` | `734759` <br> `Villa Domínguez Paulo Adrián` | `733773`

**Fecha:** Mayo 2023
    
**Profesor:** Óscar David Jaramillo Zuluaga.
    
**Link Github**: [github.com](https://github.com/paucardenasg/Proyecto_MNLP)

# <font color='maroon'> Proyecto Final </font>

In [1]:
# Librerías
import numpy as np
import pandas as pd

In [2]:
# Carga de datos
data = pd.read_csv('./Data/dataset.csv')
# Eliminando la columna de index y Unit
data.drop(columns=['SN', 'Unit'], inplace=True)
# Poniendo el date a formato de fecha
data['Date'] = pd.to_datetime(data['Date'])
# Poniendo la fecha como str
data['Fecha_Str'] = data['Date'].dt.strftime('%Y-%m-%d')
# Cambiando el nombre de las columnas a español
data.rename(columns={'Date':'Fecha', 'Commodity':'Producto', 'Minimum':'Mínimo', 'Maximum':'Máximo', 'Average':'Promedio'}, inplace=True)

# --------------------------------------------------------------------------------------------------------------------

# Cargando el dataset del cambio de Rupia Nepali a Pesos
data_cambio = pd.read_excel('./Data/CambioMoneda.xlsx')
# Poniendo Fecha en formato de fech
data_cambio['Fecha'] = pd.to_datetime(data_cambio['Fecha'])
# Quitando la hora de la fecha
data_cambio['Fecha'] = data_cambio['Fecha'].dt.date
# Poniendo la fecha como str
data_cambio['Fecha_Str'] = data_cambio['Fecha'].astype(str)

# --------------------------------------------------------------------------------------------------------------------

# Obtener las clases de productos
productos_clase = data[['Producto']]
productos_clase = productos_clase.drop_duplicates()
# Tomar solo la primera palabra del valor de la columna de producto
productos_clase['Clase'] = productos_clase['Producto'].str.split(' ').str[0]
productos_clase['Clase'] = productos_clase['Clase'].str.split('(').str[0]

# Mergeando los datos con la clase de producto
data = data.merge(productos_clase, on='Producto')

# --------------------------------------------------------------------------------------------------------------------

data = data.merge(data_cambio.drop(columns=['Fecha']), on='Fecha_Str').drop(columns=['Fecha_Str'])
# Multiplicando el precio por el cambio de moneda
data['Mínimo'] = np.round(data['Mínimo'] * data['Valor'], decimals=4)
data['Máximo'] = np.round(data['Máximo'] * data['Valor'], decimals=4)
data['Promedio'] = np.round(data['Promedio'] * data['Valor'], decimals=4)
# Eliminar la columna de valor
data.drop(columns=['Valor'], inplace=True)
# Poniendo la fecha como indice
data.set_index('Fecha', inplace=True)

# Ordenando los datos por fecha y por producto
data.sort_values(by=['Fecha', 'Producto'], inplace=True)

# Ordenando las columnas
data = data[['Producto', 'Clase', 'Mínimo', 'Máximo', 'Promedio']]

data.head()

,Producto,Clase,Mínimo,Máximo,Promedio
Fecha,,,,,
2013-06-16,Apple(Jholey),Apple,13.6746,16.4096,15.0421
2013-06-16,Asparagus,Asparagus,13.6746,20.5120,17.0933
2013-06-16,Bamboo Shoot,Bamboo,6.8373,8.2048,7.5211
2013-06-16,Banana,Banana,9.5723,10.9397,10.2560
2013-06-16,Bitter Gourd,Bitter,1.9145,2.1879,2.0512


## EDA

In [3]:
# Productos
tipos_de_productos = data['Producto'].unique()
print(f'Cantidad de Productos: {len(tipos_de_productos)}')

Cantidad de Productos: 132


In [4]:
# Porcentaje que representa cada producto
porcentaje = data['Producto'].value_counts(normalize=True).reset_index().sort_values(by='Producto', ascending=False)
# Multiplicando por 100 para obtener el porcentaje
porcentaje['Producto'] = np.round(porcentaje['Producto'] * 100, decimals=3)
# Haciendo una suma acumulativa
porcentaje['Cum_Sum'] = porcentaje['Producto'].cumsum()
porcentaje.rename(columns={'index': 'Producto', 'Producto': 'Porcentaje'}, inplace=True)
porcentaje

,Producto,Porcentaje,Cum_Sum
0,Ginger,1.386,1.386
1,Cauli Local,1.385,2.771
2,Cabbage(Local),1.385,4.156
3,Chilli Dry,1.385,5.541
4,Raddish White(Local),1.384,6.925
...,...,...,...
127,Maize,0.013,99.975
128,Mango(Calcutte),0.013,99.988
129,Sweet Lime,0.008,99.996
130,Musk Melon,0.004,100.000


In [10]:
porcentaje[porcentaje['Porcentaje'] > 0.5]

,Producto,Porcentaje,Cum_Sum
0,Ginger,1.386,1.386
1,Cauli Local,1.385,2.771
2,Cabbage(Local),1.385,4.156
3,Chilli Dry,1.385,5.541
4,Raddish White(Local),1.384,6.925
...,...,...,...
72,Bakula,0.582,85.716
73,Snake Gourd,0.566,86.282
74,Yam,0.565,86.847
75,Soyabean Green,0.527,87.374
